# Exploration du Dataset ISIC

Ce notebook effectue l'analyse exploratoire complète du dataset de dermatologie ISIC.

**Sections :**
1. Exploration de la structure des dossiers
2. Affichage des images avec Matplotlib
3. Chargement de MobileNetV2 pré-entraîné
4. Pipeline de prétraitement des images
5. Inférence sur images de test
6. Statistiques et analyse du dataset

## 1. Exploration de la structure des dossiers

In [ ]:
import os
from pathlib import Path

# définir les chemins de base
base_dir = Path("..")  # assuming notebook is in ia_vision/notebooks

# on affiche la structure des dossiers data
for sub in ["raw", "processed", "splits"]:
    path = base_dir / "data" / sub
    exists = path.exists()
    count = len(list(path.glob("*"))) if exists else 0
    print(f"{sub}: {path.resolve()} — existe: {exists}, fichiers: {count}")

# voir si des fichiers existent
print("\nContenu du répertoire racine:")
print(list(base_dir.iterdir()))

# compter les images disponibles
raw_path = base_dir / "data" / "raw"
if raw_path.exists():
    jpg_files = list(raw_path.glob("**/*.jpg"))
    png_files = list(raw_path.glob("**/*.png"))
    print(f"\nImages JPG trouvées : {len(jpg_files)}")
    print(f"Images PNG trouvées : {len(png_files)}")
else:
    print("\nDossier data/raw introuvable — placez les images ISIC dans ce dossier.")


## 2. Affichage des images avec Matplotlib

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

sample_path = base_dir / "data" / "raw"
image_files = []
if sample_path.exists():
    image_files = list(sample_path.glob("*.jpg")) + list(sample_path.glob("*.png"))

if image_files:
    n_show = min(6, len(image_files))
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    axes = axes.flatten()
    for i in range(n_show):
        img = Image.open(image_files[i])
        axes[i].imshow(img)
        axes[i].axis('off')
        axes[i].set_title(image_files[i].name, fontsize=8)
    for j in range(n_show, len(axes)):
        axes[j].axis('off')
    plt.suptitle("Exemples d'images ISIC", fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("Pas d'images disponibles dans data/raw — affichage ignoré.")


## 3. Chargement de MobileNetV2 pré-entraîné

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms

# Charger MobileNetV2 pré-entraîné sur ImageNet
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
model.eval()

print("MobileNetV2 chargé avec succès.")
print(f"Nombre de paramètres : {sum(p.numel() for p in model.parameters()):,}")

# Pipeline de transformation standard ImageNet
preprocess_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

print("\nArchitecture du classifieur :")
print(model.classifier)


## 4. Pipeline de prétraitement des images

In [ ]:
import sys
sys.path.insert(0, str(base_dir / "src"))

from preprocess import ImagePreprocessor

preprocessor = ImagePreprocessor(target_size=(224, 224))

if image_files:
    sample_img_path = str(image_files[0])
    processed = preprocessor.preprocess(sample_img_path)
    print(f"Image source : {image_files[0].name}")
    print(f"Shape après prétraitement : {processed.shape}")
    print(f"Min / Max des valeurs : {processed.min():.3f} / {processed.max():.3f}")

    # Afficher original vs prétraité
    original = preprocessor.load_image(sample_img_path)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    ax1.imshow(original)
    ax1.set_title("Original")
    ax1.axis('off')
    ax2.imshow(processed)
    ax2.set_title("Prétraité (CLAHE + normalisé)")
    ax2.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("Pas d'images disponibles — démo du prétraitement ignorée.")
    print(f"ImagePreprocessor instancié avec target_size={preprocessor.target_size}")


## 5. Inférence sur images de test

In [ ]:
import json
import urllib.request

# Charger les labels ImageNet
IMAGENET_LABELS_URL = (
    "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels"
    "/master/imagenet-simple-labels.json"
)

try:
    with urllib.request.urlopen(IMAGENET_LABELS_URL, timeout=5) as resp:
        imagenet_labels = json.loads(resp.read())
except Exception:
    imagenet_labels = [f"class_{i}" for i in range(1000)]
    print("Labels ImageNet non disponibles hors-ligne — utilisation de noms génériques.")

if image_files:
    img_pil = Image.open(image_files[0]).convert("RGB")
    input_tensor = preprocess_transform(img_pil).unsqueeze(0)  # (1, 3, 224, 224)

    with torch.no_grad():
        output = model(input_tensor)

    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    top5_prob, top5_idx = torch.topk(probabilities, 5)

    print(f"Inférence sur : {image_files[0].name}\n")
    print("Top-5 prédictions ImageNet (indicatif — modèle non fine-tuné sur ISIC) :")
    for prob, idx in zip(top5_prob, top5_idx):
        label = imagenet_labels[idx.item()] if idx.item() < len(imagenet_labels) else f"class_{idx.item()}"
        print(f"  {label:40s} {prob.item()*100:.2f}%")
else:
    print("Pas d'images disponibles — inférence ignorée.")


## 6. Statistiques et analyse du dataset

In [ ]:
import pandas as pd
import numpy as np

stats = []
if image_files:
    for img_path in image_files[:50]:  # limiter à 50 images pour la démo
        img = preprocessor.load_image(str(img_path))
        h, w, c = img.shape
        stats.append({
            "filename": img_path.name,
            "width": w,
            "height": h,
            "mean_r": img[:, :, 0].mean(),
            "mean_g": img[:, :, 1].mean(),
            "mean_b": img[:, :, 2].mean(),
        })

    df = pd.DataFrame(stats)
    print(f"Nombre d'images analysées : {len(df)}")
    print("\nStatistiques des dimensions :")
    print(df[["width", "height"]].describe())
    print("\nMoyennes des canaux RGB :")
    print(df[["mean_r", "mean_g", "mean_b"]].describe())

    # Distribution des tailles
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].hist(df["width"], bins=20, color='steelblue', edgecolor='white')
    axes[0].set_title("Distribution des largeurs")
    axes[0].set_xlabel("Pixels")
    axes[0].set_ylabel("Fréquence")

    axes[1].bar(["R", "G", "B"],
                [df["mean_r"].mean(), df["mean_g"].mean(), df["mean_b"].mean()],
                color=["#e74c3c", "#2ecc71", "#3498db"])
    axes[1].set_title("Moyenne des canaux RGB")
    axes[1].set_ylabel("Valeur (0-255)")
    plt.tight_layout()
    plt.show()
else:
    print("Pas d'images disponibles — statistiques non calculées.")
    print("Placez les images ISIC dans ia_vision/data/raw/ pour activer cette section.")
